In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [9]:
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.utils.math import cosine_similarity

model = AzureChatOpenAI(temperature=0.3,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

In [4]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""


In [7]:
embeddings = OpenAIEmbeddings(chunk_size=10)
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [30]:
query = "What is the speed of light?"
query_embedding = embeddings.embed_query(query)
similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
print(similarity)

[[0.77048288 0.74227128]]


In [29]:
cosine_similarity([query_embedding], prompt_embeddings)

array([[0.77048288, 0.74227128]])

In [26]:
prompt_templates[similarity.argmax()]

"You are a very smart physics professor. You are great at answering questions about physics in a concise and easy to understand manner. When you don't know the answer to a question you admit that you don't know.\n\nHere is a question:\n{query}"

In [8]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


In [10]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | model
    | StrOutputParser()
)

In [11]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is extremely strong, so strong that nothing, not even light, can escape its gravitational pull. It is formed when a massive star collapses under its own gravity after running out of nuclear fuel. The collapse is so intense that it creates a singularity, a point of infinite density at the center of the black hole. The boundary of the black hole, called the event horizon, marks the point of no return, beyond which anything that enters is trapped forever. Black holes have fascinated scientists for decades and continue to be a subject of ongoing research and exploration.


In [12]:
print(chain.invoke("What's a path integral"))

Using MATH
Thank you for your kind words! I'll do my best to explain what a path integral is.

In mathematics and physics, a path integral is a concept used in the field of calculus of variations and quantum mechanics. It is a way of calculating the probability amplitude or the expectation value of a physical quantity by summing over all possible paths that a system can take.

To understand this concept, let's consider a simple example. Imagine a particle moving from point A to point B in space. In classical mechanics, we would typically calculate the path of the particle by solving the equations of motion. However, in quantum mechanics, particles can exhibit wave-like behavior and can take multiple paths simultaneously.

The path integral approach accounts for this wave-like behavior by considering all possible paths the particle could take. Each path is assigned a probability amplitude, which is a complex number representing the likelihood of the particle taking that specific path. T